# For WZE-UAV Folder Structure: 
# SAPOS query + REDToolBox PPK tagging pipeline
This Jupyter notebook is an interactive wrapper around the original **`sapos_query.py`** script.  
It lets you generate the SAPOS query string for **DJI Phantom 4 Multispectral** datasets:

1. **Point the notebook at your data folder** (the one you would normally pass to the script).
2. Run the cell.
3. It prints the SAPOS query string **and** writes `@sapos_query.txt` in that same folder.
4. Generates a REDToolBox Query for PPK geotagging of P4M imagery
5. Copys corrected images to a separate folder

> **Tip:** afterwards the uncorrected images can be deleted to save storage!

---

In [10]:
import os
import re
import time
from datetime import datetime, timedelta
from math import ceil

import pytz
#from modules.platform import * #import for platforms such as DJI L2 or Wingtra data
#from modules.sapos_query import * #import sapos query
from modules.sapos_batch import batch_generate_sapos_queries_v2
from modules.rename_rinex_tool import *
from modules.wze_uav import extract_fplans, copy_vrs_for_fplans, generate_redtoolbox_batch, copy_ppk_images

# exifread is only required if you extend the notebook to read EXIF data
# import exifread


### Generate SAPOS Query

In [11]:
images_dir = r"D:\Drohnendaten\10_WZE-UAV\2024\TEMP" # path to the wze-uav main folder
sapos_out = r"D:\Drohnendaten\10_WZE-UAV\2024"

batch_generate_sapos_queries_v2(
    root_dir   = images_dir,
    master_out = sapos_out,  
    recurse    = True
)

🛸 Detected DJI Phantom 3 Multispectral dataset (v2)
📄 0,0   0,0   0   07.08.2024   08:18:44   31   1   R3   16197
✅ SAPOS query written to D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\@sapos_query.txt
✅ 20240807\16197

📝  1 query line(s) saved to D:\Drohnendaten\10_WZE-UAV\2024\all_sapos_queries_v2.txt


### Generate REDToolBox Query

In [12]:
sapos_fn = r'D:\Drohnendaten\10_WZE-UAV\2024\all_sapos_queries_v2.txt' # path to generated SAPOS query .txt
rtb_fn = r'D:\Drohnendaten\10_WZE-UAV\2024\redtoolbox_wze2024_temp.txt' # path where the REDToolBox .txt should be saved
vrs_dir = r"D:\Drohnendaten\10_WZE-UAV\2024\02_SAPOS" # path where the SAPOS files were saved to

extract_fplans(sapos_fn, images_dir, rtb_fn)

Found 1 FPLAN entries.
Saved list to: D:\Drohnendaten\10_WZE-UAV\2024\redtoolbox_wze2024_temp.txt


### Copy SAPOS files into FPLAN folders

In [13]:
copy_vrs_for_fplans(rtb_fn, vrs_dir)

📄 Copied file: D:\Drohnendaten\10_WZE-UAV\2024\02_SAPOS\16197_7893220.24o -> D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\105FPLAN\16197_7893220.24o
📄 Copied file: D:\Drohnendaten\10_WZE-UAV\2024\02_SAPOS\16197_Ephm220.24p -> D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\105FPLAN\16197_Ephm220.24p
📄 Copied file: D:\Drohnendaten\10_WZE-UAV\2024\02_SAPOS\16197_Protokoll_7893220_VRS.txt -> D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\105FPLAN\16197_Protokoll_7893220_VRS.txt


### Generate .batch for REDToolBox batch geotagging

In [14]:
batch_dir = r'D:\Ecke_Simon\Redtoolbox_v4\batch'
log_dir = r'D:\Ecke_Simon\Redtoolbox_v4\log'
generate_redtoolbox_batch(rtb_fn, batch_dir, log_dir, epn_yr='24') #set epn year to the year when the images were collected

Starting batch generation
Processing D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\105FPLAN
    Found MRK file: 105_Timestamp.MRK
    Found OBS file: 105_Rinex.obs
    Found O file: 16197_7893220.24o
    Found P file: 16197_Ephm220.24p

Total time elapsed: 0:00:01.240335
Batch file written to: D:\Ecke_Simon\Redtoolbox_v4\batch\REDToolBox_CMD_Start_batch_ALL_2025-06-11_110131.bat


### Now Windows Powershell needs to be used to do the REDToolBox PPK geotagging batch processing

### Afterwards copy PPK images and MEDIA folders to a separate location

In [16]:
src = r'D:\Drohnendaten\10_WZE-UAV\2024\TEMP'
dst = r'D:\Drohnendaten\10_WZE-UAV\2024\PPK_TEMP'
copy_ppk_images(src, dst)

Copied: D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\104MEDIA2 -> D:\Drohnendaten\10_WZE-UAV\2024\PPK_TEMP\20240807\16197\104MEDIA2
Copied: D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\106MEDIA -> D:\Drohnendaten\10_WZE-UAV\2024\PPK_TEMP\20240807\16197\106MEDIA
Copied: D:\Drohnendaten\10_WZE-UAV\2024\TEMP\20240807\16197\105FPLAN\out\EXIF_images -> D:\Drohnendaten\10_WZE-UAV\2024\PPK_TEMP\20240807\16197\105FPLAN\out\EXIF_images
